#### Cross Check of Outputs in MFLAMPA: <MH/Distr><1D/2D/Time>

##### Step 0: Package

In [1]:
#!/usr/bin/env python3
import os
import re
import numpy as np

##### Step 1: Get the Outputs Ready

##### Step 1(a): Run the test (test_spectra)

In [2]:
#==============================================================================
def run_test(testname, IsPrintRunlog=False):
    
    # Go to the directory in SP/MFLAMPA/
    dirCurr = os.getcwd()
    strdir = "SP/MFLAMPA"
    dirCurrId = dirCurr.index(strdir)
    if(dirCurrId < 0):
        print("Warning: Incorrect directory! Check the path!")
    dirCurr = dirCurr[0:dirCurrId+len(strdir)]
    os.chdir(dirCurr)
    print("Current dir:", os.getcwd())
    
    # Run the test for cross validation
    TypeTestName = type(testname)
    if(TypeTestName != str):
        print("Warning: Incorrect test name! Re-enter the test name!")
    # os.popen("csh -c 'nagfor -xlicinfo'")
    log_test = os.popen(\
        "csh -c 'make test_%s TESTDIR=run_%s'"%(testname, testname))
    if(IsPrintRunlog): print("%s runlog: %s"%(testname, log_test.read()))
#==============================================================================

run_test(testname="spectra")

Current dir: /Users/whliu/test3/SWMF/SP/MFLAMPA


ar: creating archive libMFLAMPA.a
NAG Fortran Compiler Release 7.1(Hanzomon) Build 7138


##### Step 1(b): Load All the Outputs

In [3]:
nLon = 4; nLat = 4
#==============================================================================
def read_line(filename, linenum):
    # Given filename, read No.{linenum} in this file, and return string
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            return lines[linenum - 1].strip()
    except FileNotFoundError:
        return f"File {filename} not found."
    except IndexError:
        return f"File {filename} does not have {filename} lines."
    except Exception as e:
        return f"An error occurred: {e}"
#==============================================================================
def read_head(filename, lineVars, lineInfo):
    # Given filename, read nVARs and VARs name

    # read VARs/cols name
    VarsName = read_line(filename, lineVars)
    VarsName = VarsName.split(" ")
    for iVar in range(len(VarsName)-1, -1, -1):
        if VarsName[iVar] == "":
            del VarsName[iVar]

    # read file number, simulation time, ncols
    fileInfo = read_line(filename, lineInfo).split(" ")
    for iInfo in range(len(fileInfo)-1, -1, -1):
        if fileInfo[iInfo] == "":
            del fileInfo[iInfo]
    nVars = int(fileInfo[2]) + int(fileInfo[4])

    # Get VARs Name
    VarsName = VarsName[0:nVars]
    return nVars, VarsName
#==============================================================================
def load_outputs(testname):
    # Load all test outputs: <MH/Distr><1D/2D/Time>

    lineMhVars = 5; lineDistrVars = 4; lineInfo = 2
    #==========================================================================
    def read_mh1d(IsPrintFileName=False):
        # Read the MH1D data and turn them into a dictionary

        if len(fileMh1D_I) < 1:
            print("read_mh1d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMh1D_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mh1d = {}
        for Var in ['Time'] + VarsName:
            dic_mh1d[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileMh1D_I)):
            fileName = fileMh1D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[8:11])  # 1-based
            iLat = int(fileName[12:15]) # 1-based

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_mh1d['Time'][iLon-1][iLat-1].append(float(fileInfo[1]))

            # Add data
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mh1d[VarsName[iVar]][iLon-1][iLat-1].append(data[iVar])

        # Return dic_mh1d, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: iIter, at SPTime
        # Index 5: LagrID (only for VARs)
        print("finish read_mh1d")
        return dic_mh1d
    #==========================================================================
    def read_mh2d(IsPrintFileName=False):
        # Read the MH2D data and turn them into a dictionary

        if len(fileMh2D_I) < 1:
            print("read_mh2d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMh2D_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mh2d = {}
        for Var in ['Time'] + VarsName:
            dic_mh2d[Var] = []

        # Read data from files
        for iFile in range(len(fileMh2D_I)):
            fileName = fileMh2D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_mh2d['Time'].append(float(fileInfo[1]))
            
            # Add data
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mh2d[VarsName[iVar]].append(data[iVar])

        # Return dic_mh2d, structured by
        # Index 1: VAR name
        # Index 2: iIter, at SPTime
        # Index 3: iLine (only for VARs)
        print("finish read_mh2d")
        return dic_mh2d
    #==========================================================================
    def read_mhtime(IsPrintFileName=False):
        # Read the MHTime data and turn them into a dictionary

        if len(fileMhTime_I) < 1:
            print("read_mhtime: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMhTime_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mhtime = {}
        for Var in VarsName:
            dic_mhtime[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileMhTime_I)):
            fileName = fileMhTime_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[-11:-8]) # 1-based
            iLat = int(fileName[-7:-4])  # 1-based

            # Add data (including time)
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mhtime[VarsName[iVar]][iLon-1][iLat-1].append(data[iVar])

        # Return dic_mhtime, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: 0, since we only append once
        # Index 5: iIter, at SPTime
        print("finish read_mhtime")
        return dic_mhtime
    #==========================================================================
    def read_distr1d(fileDistr1D_I, IsPrintFileName=False):
        # Read the Distr1D data and turn them into a dictionary

        if len(fileDistr1D_I) < 1:
            print("read_distr1d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistr1D_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distr1d = {}
        for Var in ['Time'] + VarsName:
            dic_distr1d[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileDistr1D_I)):
            fileName = fileDistr1D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[12:15]) # 1-based
            iLat = int(fileName[16:19]) # 1-based

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_distr1d['Time'][iLon-1][iLat-1].append(float(fileInfo[1]))

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nX = int(fileDim[1])

            # Add data
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distr1d[VarsName[iVar]][iLon-1][iLat-1].append( \
                    data[iVar].reshape(nX, nP))

        # Return dic_distr1d, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: iIter, since we only append once
        # Index 5: LagrID from 1 to iEnd, for the spectrum
        # Index 6: iMomentum/iEnergy, for the spectrum
        print("finish read_distr1d")
        return dic_distr1d
    #==========================================================================
    def read_distr2d(fileDistr2D_I, IsPrintFileName=False):
        # Read the Distr2D data and turn them into a dictionary

        if len(fileDistr2D_I) < 1:
            print("read_distr2d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistr2D_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distr2d = {}
        for Var in ['Time'] + VarsName:
            dic_distr2d[Var] = []

        # Read data from files
        for iFile in range(len(fileDistr2D_I)):
            fileName = fileDistr2D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_distr2d['Time'].append(float(fileInfo[1]))

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nX = int(fileDim[1])

            # Add data
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distr2d[VarsName[iVar]].append( \
                    data[iVar].reshape(nX, nP))

        # Return dic_distr2d, structured by
        # Index 1: VAR name
        # Index 2: iIter
        # Index 3: LagrID, from 1 to iEnd for the spectrum (only for VARs)
        # Index 4: iMomentum/iEnergy, for the spectrum (only for VARs)
        print("finish read_distr2d")
        return dic_distr2d
    #==========================================================================
    def read_distrtime(fileDistrTime_I, IsPrintFileName=False):
        # Read the DistrTime data and turn them into a dictionary

        if len(fileDistrTime_I) < 1:
            print("read_distrtime: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistrTime_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distrtime = {}
        for Var in VarsName:
            dic_distrtime[Var] = \
                [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileDistrTime_I)):
            fileName = fileDistrTime_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[-11:-8]) # 1-based
            iLat = int(fileName[-7:-4])  # 1-based

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nTime = int(fileDim[1])

            # Add data (including time)
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distrtime[VarsName[iVar]][iLon-1][iLat-1].append( \
                    data[iVar].reshape(nTime, nP))

        # Return dic_distrtime, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: 0, since we only append once
        # Index 5: iIter, for the spectrum at SPTime
        # Index 6: iMomentum/iEnergy, for the spectrum
        print("finish read_distrtime")
        return dic_distrtime
    #==========================================================================

    # Go to the directory in SP/MFLAMPA/
    dirCurr = os.getcwd()
    strdir = "SP/MFLAMPA"
    dirCurrId = dirCurr.index(strdir)
    if(dirCurrId < 0):
        print("Warning: Incorrect directory! Check the path!")
    dirCurr = dirCurr[0:dirCurrId+len(strdir)]
    os.chdir(dirCurr)
    print("Current dir:", os.getcwd())

    # Get the path and all the output file names
    path = "./run_%s/SP/IO2/"%(testname)
    filelist = sorted(os.listdir(path))

    # Declare the arrays the save different types of outputs
    fileMh1D_I   = []; fileDistr1DcdfS_I  = []; fileDistr1DdefR_I = []
    fileMh2D_I   = []; fileDistr2Ddef_I   = []
    fileMhTime_I = []; fileDistrTimedef_I = []
    # Declare the file name format for different types of outputs
    formatMH1D   = re.compile( \
        r"^MH_data_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatMH2D   = re.compile( \
        r"^MH_data_R=\d{4}\.\d{2}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatMHTime = re.compile( \
        r"^MH_data_R=\d{4}\.\d{2}_\d{3}_\d{3}.out$")
    formatDistr1DcdfS = re.compile( \
        r"^Distr_cdf_S_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistr1DdefR = re.compile( \
        r"^Distr_def_R_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistr2Ddef     = re.compile( \
        r"^Distr_def_R=\d{4}\.\d{2}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistrTimedef   = re.compile( \
        r"^Distr_def_R=\d{4}\.\d{2}_\d{3}_\d{3}.out$")

    # Separate different types of outputs to the corresponding lists
    for iFile in range(len(filelist)):
        if formatMH1D.match(filelist[iFile]):
            fileMh1D_I.append(filelist[iFile])
        elif formatMH2D.match(filelist[iFile]):
            fileMh2D_I.append(filelist[iFile])
        elif formatMHTime.match(filelist[iFile]):
            fileMhTime_I.append(filelist[iFile])
        elif formatDistr1DcdfS.match(filelist[iFile]):
            fileDistr1DcdfS_I.append(filelist[iFile])
        elif formatDistr1DdefR.match(filelist[iFile]):
            fileDistr1DdefR_I.append(filelist[iFile])
        elif formatDistr2Ddef.match(filelist[iFile]):
            fileDistr2Ddef_I.append(filelist[iFile])
        elif formatDistrTimedef.match(filelist[iFile]):
            fileDistrTimedef_I.append(filelist[iFile])

    # Read MH data and Distr data
    dic_mh1d   = read_mh1d(IsPrintFileName=False)
    dic_mh2d   = read_mh2d(IsPrintFileName=False)
    dic_mhtime = read_mhtime(IsPrintFileName=False)
    dic_distr1dcdfS  = read_distr1d(fileDistr1DcdfS_I, False)
    dic_distr1ddefR  = read_distr1d(fileDistr1DdefR_I, False)
    dic_distr2ddef   = read_distr2d(fileDistr2Ddef_I, False)
    dic_distrtimedef = read_distrtime(fileDistrTimedef_I, False)

    # Organize all test outputs into one directories
    dicOut = {}
    dicOut['MH1D']   = dic_mh1d
    dicOut['MH2D']   = dic_mh2d
    dicOut['MHTime'] = dic_mhtime
    dicOut['Distr1DcdfS']  = dic_distr1dcdfS
    dicOut['Distr1DdefR']  = dic_distr1ddefR
    dicOut['Distr2Ddef']   = dic_distr2ddef
    dicOut['DistrTimedef'] = dic_distrtimedef
    return dicOut
#==============================================================================

dicSpectra = load_outputs(testname="spectra")

Current dir: /Users/whliu/test3/SWMF/SP/MFLAMPA


finish read_mh1d
finish read_mh2d
finish read_mhtime
finish read_distr1d
finish read_distr1d
finish read_distr2d
finish read_distrtime


##### Step 2: Check MH_data (MH1D, MH2D, MHTime)

In [4]:
Radius = 215.0
#==============================================================================
def rebin_data(data, binning, order=1):

    # interpolate y = y(x) (in data= (x,y)) to new x

    # Parmeters
    # ---------
    # data, 2-d float array. data[1] -- y, data[0] --x 
    # binning, float array, the new x binning 

    # Returns
    # -------
    # float array, same length as binning

    import numpy as np
    from scipy.interpolate import InterpolatedUnivariateSpline
    from scipy.interpolate import interp1d
    func = interp1d(data[0], data[1])

    # x must be strictly increasing
    while np.min(np.diff(data[0]))<=0:
        msk1= np.append([True], np.diff(data[0])>0)
        data[0]=data[0][msk1]
        data[1]=data[1][msk1]
    # y must not be nan values
    msk2 = ~np.isnan(data[1])
    temp = np.array(data)
    func = InterpolatedUnivariateSpline(temp[0][msk2], temp[1][msk2], k=order)
    return func(binning)
#==============================================================================
def check_MHdata(dicOut, testname):
    # Cross-check MH Data in different formats

    #==========================================================================
    def check_MH1d2d():
        # Check MH1D and MH2D data
        dic_mh1dto2d = {}
        DiffAmaxMh1dto2d = 0.0; DiffRmaxMh1dto2d = 0.0

        for iVar in range(len(keys_mh2d)):
            dic_mh1dto2d[keys_mh2d[iVar]] = []
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                # Check Difference in Time
                DiffAmaxMh1dto2d = max(DiffAmaxMh1dto2d, np.max(np.abs( \
                    np.array(dic_mh1d['Time'][iLon-1][iLat-1][1:]) - \
                    np.array(dic_mh2d['Time']))))
                DiffRmaxMh1dto2d = max(DiffRmaxMh1dto2d, np.max(np.abs(( \
                    np.array(dic_mh1d['Time'][iLon-1][iLat-1][1:]) - \
                    np.array(dic_mh2d['Time'])) / \
                    np.array(dic_mh2d['Time']))))
                # Get nTime
                nTime = len(dic_mh1d['Time'][iLon-1][iLat-1][1:])

        # Iterate over Time, convert MH1D => MH2D
        for iTime in range(1, nTime+1):
            dataAtTime = [[] for _ in range(len(keys_mh2d)-1)]
            for iLat in range(1, nLon+1):
                for iLon in range(1, nLat+1):
                    # Get LineIndex
                    iLineAll = (iLat-1)*nLat + iLon
                    dataAtTime[0].append(iLineAll)
                    # Rebin for other saved variables
                    Radius1d = np.sqrt( 
                        np.array(dic_mh1d['X'][iLon-1][iLat-1][iTime])**2 + \
                        np.array(dic_mh1d['Y'][iLon-1][iLat-1][iTime])**2 + \
                        np.array(dic_mh1d['Z'][iLon-1][iLat-1][iTime])**2)
                    for iVar in range(2, len(keys_mh2d)):
                        if iVar == 5 or iVar == 6: continue
                        NameVar = keys_mh2d[iVar]
                        data1d = dic_mh1d[NameVar][iLon-1][iLat-1][iTime]
                        data1dre = rebin_data([Radius1d, data1d], [Radius])[0]
                        dataAtTime[iVar-1].append(data1dre)

            # Assign to dic_mh1dto2d for comparison
            for iVar in range(1, len(keys_mh2d)):
                if iVar == 5 or iVar == 6: continue
                NameVar = keys_mh2d[iVar]
                dic_mh1dto2d[NameVar].append(dataAtTime[iVar-1])
                # Check Difference in other variables
                DiffAmaxMh1dto2d = max(DiffAmaxMh1dto2d, np.max(np.abs( \
                    np.array(dic_mh1dto2d[NameVar][iTime-1]) - \
                    np.array(dic_mh2d[NameVar][iTime-1]))))
                DiffRmaxMh1dto2d = max(DiffRmaxMh1dto2d, np.max(np.abs(( \
                    np.array(dic_mh1dto2d[NameVar][iTime-1]) - \
                    np.array(dic_mh2d[NameVar][iTime-1])) / \
                    np.array(dic_mh2d[NameVar][iTime-1]))))

        # Return maximum absolute and relative differences
        return DiffAmaxMh1dto2d, DiffRmaxMh1dto2d
    #==========================================================================
    def check_MH1dtime():
        # Check MH1D and MHTime data
        dic_mh1dtotime = {}
        DiffAmaxMh1dtotime = 0.0; DiffRmaxMh1dtotime = 0.0

        for iVar in range(len(keys_mhtime)):
            dic_mh1dtotime[keys_mhtime[iVar]] = \
                [[[] for _ in range(nLat)] for _ in range(nLon)]
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                # Check Difference in Time
                DiffAmaxMh1dtotime = max(DiffAmaxMh1dtotime, np.max(np.abs( \
                    np.array(dic_mh1d['Time'][iLon-1][iLat-1][1:]) - \
                    np.array(dic_mhtime['Time'][iLon-1][iLat-1]))))
                DiffRmaxMh1dtotime = max(DiffRmaxMh1dtotime, np.max(np.abs(( \
                    np.array(dic_mh1d['Time'][iLon-1][iLat-1][1:]) - \
                    np.array(dic_mhtime['Time'][iLon-1][iLat-1])) / \
                    np.array(dic_mhtime['Time'][iLon-1][iLat-1]))))
                # Get nTime
                nTime = len(dic_mh1d['Time'][iLon-1][iLat-1][1:])

        # Iterate over iLon, iLat, and Time, convert MH1D => MHTime
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                dataAtTime = [[] for _ in range(len(keys_mhtime)-1)]
                for iTime in range(1, nTime+1):
                    # Rebin for other saved variables
                    Radius1d = np.sqrt( 
                        np.array(dic_mh1d['X'][iLon-1][iLat-1][iTime])**2 + \
                        np.array(dic_mh1d['Y'][iLon-1][iLat-1][iTime])**2 + \
                        np.array(dic_mh1d['Z'][iLon-1][iLat-1][iTime])**2)
                    for iVar in range(1, len(keys_mhtime)):
                        NameVar = keys_mhtime[iVar]
                        data1d = dic_mh1d[NameVar][iLon-1][iLat-1][iTime]
                        data1dre = rebin_data([Radius1d, data1d], [Radius])[0]
                        dataAtTime[iVar-1].append(data1dre)

                # Assign to dic_mh1dtotime for comparison
                for iVar in range(1, len(keys_mhtime)):
                    NameVar = keys_mhtime[iVar]
                    dic_mh1dtotime[NameVar][iLon-1][iLat-1].append( \
                        dataAtTime[iVar-1])
                    # Check Difference in other variables
                    DiffAmaxMh1dtotime = max(DiffAmaxMh1dtotime, np.max(np.abs\
                        (np.array(dic_mh1dtotime[NameVar][iLon-1][iLat-1]) - \
                        np.array(dic_mhtime[NameVar][iLon-1][iLat-1]))))
                    DiffRmaxMh1dtotime = max(DiffRmaxMh1dtotime, np.max(np.abs\
                        ((np.array(dic_mh1dtotime[NameVar][iLon-1][iLat-1]) - \
                        np.array(dic_mhtime[NameVar][iLon-1][iLat-1])) / \
                        np.array(dic_mhtime[NameVar][iLon-1][iLat-1]))))

        # Return maximum absolute and relative differences
        return DiffAmaxMh1dtotime, DiffRmaxMh1dtotime
    #==========================================================================

    # Get MH1D/MH2D/MHTime Outputs
    dic_mh1d   = dicOut['MH1D'];   keys_mh1d   = list(dic_mh1d.keys())
    dic_mh2d   = dicOut['MH2D'];   keys_mh2d   = list(dic_mh2d.keys())
    dic_mhtime = dicOut['MHTime']; keys_mhtime = list(dic_mhtime.keys())
    print("VARs in mh1d:", keys_mh1d)
    print("VARs in mh2d:", keys_mh2d)
    print("VARs in mhtime:", keys_mhtime)

    # Check the Differences in Outputs
    DiffAmaxmh1dto2d, DiffRmaxmh1dto2d = check_MH1d2d()
    print("Diff in mh1d & mh2d -a=%.6e, -r=%.6e"% \
        (DiffAmaxmh1dto2d, DiffRmaxmh1dto2d))
    DiffAmaxmh1dtotime, DiffRmaxmh1dtotime = check_MH1dtime()
    print("Diff in mh1d & mhtime -a=%.6e, -r=%.6e"% \
        (DiffAmaxmh1dtotime, DiffRmaxmh1dtotime))
#==============================================================================

check_MHdata(dicSpectra, testname="spectra")

VARs in mh1d: ['Time', 'LagrID', 'X', 'Y', 'Z', 'Rho', 'T', 'Ux', 'Uy', 'Uz', 'Bx', 'By', 'Bz', 'Wave1', 'Wave2', 'flux_total', 'flux_00000005', 'flux_00000010', 'flux_00000030', 'flux_00000050', 'flux_00000060', 'flux_00000100', 'eflux']
VARs in mh2d: ['Time', 'LineIndex', 'X', 'Y', 'Z', 'Longitude', 'Latitude', 'flux_total', 'flux_00000005', 'flux_00000010', 'flux_00000030', 'flux_00000050', 'flux_00000060', 'flux_00000100', 'eflux']
VARs in mhtime: ['Time', 'X', 'Y', 'Z', 'flux_total', 'flux_00000005', 'flux_00000010', 'flux_00000030', 'flux_00000050', 'flux_00000060', 'flux_00000100', 'eflux']
Diff in mh1d & mh2d -a=8.441857e-07, -r=8.907233e-11
Diff in mh1d & mhtime -a=8.441857e-07, -r=8.907233e-11


##### Step 3: Check Distr_data (Distr1D, Distr2D, DistrTime)

In [5]:
cLightSpeed    = 2.9979E+08
cProtonMass    = 1.6726E-27
cElementCharge = 1.6022E-19
ckeV           = 1.0E+3*cElementCharge
cRmeProton     = cProtonMass*cLightSpeed**2
cEnergyInj     = 10.0*ckeV
cMomentumInj   = np.sqrt(cEnergyInj*(cEnergyInj+2*cRmeProton))/cLightSpeed
Io2SiFlux      = 10000.0
#==============================================================================
def check_Distrdata(dicOut, testname):
    # Cross-check Distr Data in different formats

    #==========================================================================
    def check_Distr1d1d():
        # Check Distr1DcdfS and Distr1DdefR data
        dic_distr1dto1d = {}
        DiffAmaxdistr1dto1d = 0.0; DiffRmaxdistr1dto1d = 0.0

        # Only for Axes 1 and 3
        for iVar in [1, 3]:
            dic_distr1dto1d[keys_distr1ddefR[iVar]] = \
                [[[] for _ in range(nLat)] for _ in range(nLon)]
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                # Check Difference in Time
                DiffAmaxdistr1dto1d = max(DiffAmaxdistr1dto1d, np.max(np.abs(\
                    np.array(dic_distr1dcdfS['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distr1ddefR['Time'][iLon-1][iLat-1]))))
                DiffRmaxdistr1dto1d = max(DiffRmaxdistr1dto1d, np.max(np.abs((\
                    np.array(dic_distr1dcdfS['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distr1ddefR['Time'][iLon-1][iLat-1])) / \
                    np.array(dic_distr1ddefR['Time'][iLon-1][iLat-1]))))
                # Get nTime
                nTime = len(dic_distr1dcdfS['Time'][iLon-1][iLat-1])

        # Iterate over iLon, iLat, and Time, convert MH1D => MHTime
        for iLat in range(1, nLat+1):
            for iLon in range(1, nLon+1):
                dataAtTime = [[], []]
                for iTime in range(1, nTime+1):
                    # Convert Momentum to Energy
                    dataAtTime[0].append(np.log10((np.sqrt(cRmeProton**2 + \
                        (cMomentumInj* 10.0**np.array(dic_distr1dcdfS \
                        [keys_distr1dcdfS[1]][iLon-1][iLat-1][iTime-1]) \
                        *cLightSpeed)**2)-cRmeProton) / ckeV))
                    # Convert cdf to def
                    dataAtTime[1].append(-np.log10(Io2SiFlux) + 2.0* \
                        np.array(dic_distr1dcdfS[keys_distr1dcdfS[1]] \
                        [iLon-1][iLat-1][iTime-1]) + \
                        np.array(dic_distr1dcdfS[keys_distr1dcdfS[3]] \
                        [iLon-1][iLat-1][iTime-1]))

                # Assign to dic_distr1dto1d for comparison
                # Only for Log10Energy (Axis 1) & Log10DiffEnergyFlux (Axis 3)
                for iVar in [1, 3]:
                    NameVar = keys_distr1ddefR[iVar]
                    dic_distr1dto1d[NameVar][iLon-1][iLat-1] = \
                        dataAtTime[int((iVar-1)/2)]
                    for iTime in range(1, nTime+1):
                        # Check Difference in other variables
                        DiffAmaxdistr1dto1d = max(DiffAmaxdistr1dto1d, \
                            np.max(np.abs(np.array(dic_distr1dto1d \
                            [NameVar][iLon-1][iLat-1][iTime-1]) - \
                            np.array(dic_distr1ddefR \
                            [NameVar][iLon-1][iLat-1][iTime-1]))))
                        DiffRmaxdistr1dto1d = max(DiffRmaxdistr1dto1d, \
                            np.max(np.abs((np.array(dic_distr1dto1d \
                            [NameVar][iLon-1][iLat-1][iTime-1]) - \
                            np.array(dic_distr1ddefR \
                            [NameVar][iLon-1][iLat-1][iTime-1])) / \
                            np.array(dic_distr1ddefR \
                            [NameVar][iLon-1][iLat-1][iTime-1]))))

        # Return maximum absolute and relative differences
        return DiffAmaxdistr1dto1d, DiffRmaxdistr1dto1d
    #==========================================================================
    def check_Distr1d2d(dic_distr1d, dic_distr2d):
        # Check Distr1DdefR and Distr2Ddef data
        dic_distr1dto2d = {}
        DiffAmaxdistr1dto2d = 0.0; DiffRmaxdistr1dto2d = 0.0
        keys_distr1d = list(dic_distr1d.keys())
        keys_distr2d = list(dic_distr2d.keys())

        for iVar in range(1, len(keys_distr2d)):
            dic_distr1dto2d[keys_distr2d[iVar]] = []
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                # Check Difference in Time
                DiffAmaxdistr1dto2d = max(DiffAmaxdistr1dto2d, np.max(np.abs(\
                    np.array(dic_distr1d['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distr2d['Time']))))
                DiffRmaxdistr1dto2d = max(DiffRmaxdistr1dto2d, np.max(np.abs((\
                    np.array(dic_distr1d['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distr2d['Time'])) / \
                    np.array(dic_distr2d['Time']))))
                # Get nTime
                nTime = len(dic_distr1d['Time'][iLon-1][iLat-1])

        # Iterate over Time, convert Distr1D => Distr2D
        for iTime in range(1, nTime+1):
            dataAtTime = [[], [], []]
            for iLat in range(1, nLon+1):
                for iLon in range(1, nLat+1):
                    # Get size in Dim1 and Dim2
                    nX, nP = dic_distr1d[keys_distr1d[2]] \
                        [iLon-1][iLat-1][iTime-1].shape
                    # Get Energy
                    dataAtTime[0].append(np.array(dic_distr1d \
                        [keys_distr1d[1]][iLon-1][iLat-1][iTime-1][0]))
                    # Get LineIndex
                    iLineAll = (iLat-1)*nLat + iLon
                    dataAtTime[1].append( \
                        np.array([iLineAll for _ in range(nP)]))
                    # Rebin for the DiffFlux
                    dataAtTimeSpec = []
                    for iP in range(nP):
                        Radius1d = dic_distr1d[keys_distr1d[2]] \
                            [iLon-1][iLat-1][iTime-1].T[iP]
                        data1d = dic_distr1d[keys_distr1d[3]] \
                            [iLon-1][iLat-1][iTime-1].T[iP]
                        data1dre = np.log10(rebin_data( \
                            [Radius1d, 10.0**data1d], [Radius])[0])
                        dataAtTimeSpec.append(data1dre)
                    dataAtTime[2].append(np.array(dataAtTimeSpec))

            # Assign to dic_distr1dto2d for comparison
            for iVar in range(1, len(keys_distr2d)):
                NameVar = keys_distr2d[iVar]
                dic_distr1dto2d[NameVar].append(dataAtTime[iVar-1])
                # Check Difference in other variables
                DiffAmaxdistr1dto2d = max(DiffAmaxdistr1dto2d, np.max(np.abs(\
                    np.array(dic_distr1dto2d[NameVar][iTime-1]) - \
                    np.array(dic_distr2d[NameVar][iTime-1]))))
                DiffRmaxdistr1dto2d = max(DiffRmaxdistr1dto2d, np.max(np.abs((\
                    np.array(dic_distr1dto2d[NameVar][iTime-1]) - \
                    np.array(dic_distr2d[NameVar][iTime-1])) / \
                    np.array(dic_distr2d[NameVar][iTime-1]))))

        return DiffAmaxdistr1dto2d, DiffRmaxdistr1dto2d
    #==========================================================================
    def check_Distr1dtime(dic_distr1d, dic_distrtime):
        # Check Distr1DdefR and DistrTime data
        dic_distr1dtotime = {}
        DiffAmaxdistr1dtotime = 0.0; DiffRmaxdistr1dtotime = 0.0
        keys_distr1d = list(dic_distr1d.keys())
        keys_distrtime = list(dic_distrtime.keys())

        for iVar in range(len(keys_distrtime)):
            dic_distr1dtotime[keys_distrtime[iVar]] = \
                [[[] for _ in range(nLat)] for _ in range(nLon)]
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                # Check Difference in Time
                DiffAmaxdistr1dtotime = max(DiffAmaxdistr1dtotime, np.max( \
                    np.abs(np.array(dic_distr1d['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distrtime['Time'][iLon-1][iLat-1][0].T[0]))))
                DiffRmaxdistr1dtotime = max(DiffRmaxdistr1dtotime, np.max( \
                    np.abs((np.array(dic_distr1d['Time'][iLon-1][iLat-1]) - \
                    np.array(dic_distrtime['Time'][iLon-1][iLat-1][0].T[0]))/ \
                    np.array(dic_distrtime['Time'][iLon-1][iLat-1][0].T[0]))))
                # Get nTime
                nTime = len(dic_distrtime['Time'][iLon-1][iLat-1][0])

        # Iterate over iLon, iLat, and Time, convert Distr1D => DistrTime
        for iLat in range(1, nLon+1):
            for iLon in range(1, nLat+1):
                dataAtTime = [[], []]
                for iTime in range(1, nTime+1):
                    nX, nP = dic_distr1d[keys_distr1d[2]] \
                        [iLon-1][iLat-1][iTime-1].shape
                    # Get Energy
                    dataAtTime[0].append(np.array(dic_distr1d \
                        [keys_distr1d[1]][iLon-1][iLat-1][iTime-1][0]))
                    # Rebin for the DiffFlux
                    dataAtTimeSpec = []
                    for iP in range(nP):
                        Radius1d = dic_distr1d[keys_distr1d[2]] \
                            [iLon-1][iLat-1][iTime-1].T[iP]
                        data1d = dic_distr1d[keys_distr1d[3]] \
                            [iLon-1][iLat-1][iTime-1].T[iP]
                        data1dre = np.log10(rebin_data( \
                            [Radius1d, 10.0**data1d], [Radius])[0])
                        dataAtTimeSpec.append(data1dre)
                    dataAtTime[1].append(np.array(dataAtTimeSpec))

                # Assign to dic_distr1dtotime for comparison
                # Only for Log10Energy (Axis 1) & Log10DiffEnergyFlux (Axis 3)
                for iVar in [1, 3]:
                    NameVar = keys_distrtime[iVar-1]
                    dic_distr1dtotime[NameVar][iLon-1][iLat-1].append( \
                        dataAtTime[int((iVar-1)/2)])
                    # Check Difference in other variables
                    DiffAmaxdistr1dtotime = max(DiffAmaxdistr1dtotime, \
                        np.max(np.abs(np.array( \
                        dic_distr1dtotime[NameVar][iLon-1][iLat-1]) - \
                        np.array(dic_distrtime[NameVar][iLon-1][iLat-1][0]))))
                    DiffRmaxdistr1dtotime = max(DiffRmaxdistr1dtotime, \
                        np.max(np.abs((np.array( \
                        dic_distr1dtotime[NameVar][iLon-1][iLat-1]) - \
                        np.array(dic_distrtime[NameVar][iLon-1][iLat-1][0]))/ \
                        np.array(dic_distrtime[NameVar][iLon-1][iLat-1][0]))))

        return DiffAmaxdistr1dtotime, DiffRmaxdistr1dtotime
    #==========================================================================

    # Get Distr1D/Distr2D/DistrTime Outputs
    dic_distr1dcdfS  = dicOut['Distr1DcdfS']
    dic_distr1ddefR  = dicOut['Distr1DdefR']
    dic_distr2ddef   = dicOut['Distr2Ddef']
    dic_distrtimedef = dicOut['DistrTimedef']
    keys_distr1dcdfS  = list(dic_distr1dcdfS.keys())
    keys_distr1ddefR  = list(dic_distr1ddefR.keys())
    keys_distr2ddef   = list(dic_distr2ddef.keys())
    keys_distrtimedef = list(dic_distrtimedef.keys())
    print("VARs in distr1dcdfS:", keys_distr1dcdfS)
    print("VARs in distr1ddefR:", keys_distr1ddefR)
    print("VARs in distr2ddef:", keys_distr2ddef)
    print("VARs in distrtimedef:", keys_distrtimedef)

    # Check the Differences in Outputs
    DiffAmaxdistr1dto1d, DiffRmaxdistr1dto1d = check_Distr1d1d()
    print("Diff in distr1dcdfS & distr1ddefR -a=%.6e, -r=%.6e"% \
        (DiffAmaxdistr1dto1d, DiffRmaxdistr1dto1d))
    DiffAmaxdistr1dto2d, DiffRmaxdistr1dto2d = \
        check_Distr1d2d(dic_distr1ddefR, dic_distr2ddef)
    print("Diff in distr1ddefR & distr2ddef -a=%.6e, -r=%.6e"% \
        (DiffAmaxdistr1dto2d, DiffRmaxdistr1dto2d))
    DiffAmaxdistr1dtotime, DiffRmaxdistr1dtotime = \
        check_Distr1dtime(dic_distr1ddefR, dic_distrtimedef)
    print("Diff in distr1ddefR & distrtimedef -a=%.6e, -r=%.6e"% \
        (DiffAmaxdistr1dtotime, DiffRmaxdistr1dtotime))
#==============================================================================

check_Distrdata(dicSpectra, testname="spectra")

VARs in distr1dcdfS: ['Time', 'Log10Momentum', 'FieldLineLength', 'Log10Distribution']
VARs in distr1ddefR: ['Time', 'Log10Energy', 'Distance', 'Log10DiffEnergyFlux']
VARs in distr2ddef: ['Time', 'Log10Energy', 'LineIndex', 'Log10DiffEnergyFlux']
VARs in distrtimedef: ['Log10Energy', 'Time', 'Log10DiffEnergyFlux']
Diff in distr1dcdfS & distr1ddefR -a=6.000018e-10, -r=5.346454e-05
Diff in distr1ddefR & distr2ddef -a=3.780089e-09, -r=4.390218e-10
Diff in distr1ddefR & distrtimedef -a=3.780089e-09, -r=4.390218e-10


##### Step 4: Check Distr1D => MH1D Data

In [6]:
#==============================================================================
def check_DistrMH1D(dicOut, testname):
    # Cross-check Distr1D and MH1D Data

    pass
#==============================================================================

check_DistrMH1D(dicSpectra, testname="spectra")